# 07.1- Création d'un dataset de la fusion des datasets de clean metrics et clean jobs/job_events

Ce notebook génère 1 csv :

- dataset_for_preprocess.csv qui fusionne les données du dataset de metrics et jobs/job_events

Etapes : 

B) Import des datasets : 

- metrics est quasi-brut (la colonne events est fractionnée et tous les identifiants d'évènements sont des nombres)

- jobs (le dataset fusionné des données de jobs et job_events) dont chaque ligne est un job, un job est unique et il a un début et une fin

C) Réduction du dataset jobs et réductions succesives du dataset metrics (les lignes évènements avec certains identifiants)

D) Concaténation des datasets jobs et metrics (suppression des lignes inutiles : toutes celles de jobs et metrics si aucun jobId associé)


# A. Imports

## a) Librairies

In [1]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## b) Datasets

In [2]:
# source path to datasets
path = '../data/'
metrics = 'metrics/clean_merge_metrics_dataset.csv'
jobs = 'jobs/merge_raw_jobs_and_clean_jobevents_dataset.csv'

save_csv = '../data/dataset_for_preprocess_criticality_balanced_07.1.csv'

In [3]:
CRITICALITY_NULL = 'NO_EVENT'

# B. Jeux de données

### a) Metrics

In [4]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(os.path.join(path, metrics), index_col=0)
print(f'metrics dataset shape {metrics_df.shape}')

metrics dataset shape (2182421, 14)


In [5]:
# la colonne timestamp contient-elle des valeurs en double ?
metrics_df['timestamp'].duplicated().any()

True

In [6]:
metrics_df['timestamp'].min()

'2022-11-02 08:43:45.619000+00:00'

In [7]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
groupes = metrics_df.groupby('timestamp').groups
print(len(groupes.keys()))

897260


In [8]:
metrics_df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-11-02 08:43:46.921000+00:00,INFO,358,iFoil L,iFoil,Gen. 2,Total Pages Counter,25411,User,Operator,ERR,12766,18000
1,iFoil,2022-11-02 08:43:46.921000+00:00,INFO,358,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,670871,User,Operator,ERR,12766,18000
2,PLC,2022-11-02 08:43:46.845000+00:00,ERROR,358,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,3359237,User,Operator,ERR,12766,18000


### b) Jobs

In [9]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, jobs), index_col=0)
print(f'jobs dataset shape {jobs_df.shape}')

jobs dataset shape (37299, 56)


In [10]:
# la colonne started_at contient-elle des valeurs en double ?
jobs_df['started_at'].duplicated().any()

False

In [11]:
jobs_df['started_at'].min()

'2022-02-22 09:43:18.116000+00:00'

In [12]:
jobs_df.head(3)

,total_copies_job,started_at,ended_at,speed_job,paperHeight_job,paperWidth_job,total_copies_requested_job,ifoil_job,scanner_mode,bars_job,...,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,total_copies_event,varnishConsumptionVarnish_3d_event
0,6,2022-02-22 09:43:18.116000+00:00,2022-02-22 09:44:33.389000+00:00,313,483,330,6,True,3,2,...,0,0,1500,24,25,False,3,SUCCESS,6,4.585923
1,11,2022-02-22 09:45:01.304000+00:00,2022-02-22 09:46:34.929000+00:00,313,483,330,11,True,3,2,...,0,0,1500,24,25,False,3,SUCCESS,11,2.917403
2,7,2022-02-22 09:47:30.319000+00:00,2022-02-22 09:48:37.554000+00:00,313,483,330,7,True,3,2,...,0,0,1500,24,25,False,3,SUCCESS,7,0.423666


Les tailles des datasets sont déséquilibrés :

- 3510431 lignes pour metrics

- 16295 lignes pour jobs

Les dates de début sont différentes :

- '2022-04-15 05:55:06.678000+00:00' pour metrics

- '2021-06-18 09:22:46.866000+00:00' pour jobs

Le dataset metrics compte 1242037 doublons pour la colonne timestamp

# C. Equilibrage des jeux de données

## a) Réduction de jobs

In [13]:
# concordance des données de temps dans un même cadre
jobs_reduced = jobs_df[jobs_df.started_at > metrics_df.timestamp.min()]
jobs_reduced.shape

(22186, 56)

## b) Réduction de metrics

In [14]:
# on remplace les valeurs de criticité nulles par 'UNDEFINED'
# ce sont le messages de metrics qui n'ont pas d'évènements
metrics_df['criticality_events'] = metrics_df['criticality_events'].fillna(CRITICALITY_NULL)

In [15]:
# on vérifie la distribution par criticité
metrics_df['criticality_events'].value_counts()

NO_EVENT    1908025
INFO         233617
ERROR         29395
WARNING       11384
Name: criticality_events, dtype: int64

In [16]:
# on détermine le nombre de lignes ERROR
errors = metrics_df['criticality_events'][metrics_df['criticality_events'] == 'ERROR'].count()
print(f"Le nombre de lignes en criticité 'ERROR' : {errors}")

Le nombre de lignes en criticité 'ERROR' : 29395


In [17]:
# def balance_dataframe_by_criticality(df, criticality, n):
#     # on filtre les lignes du df qui ont la valeur criticality dans la colonne 'criticality_events'
#     df_criticality = df[df['criticality_events'] == criticality]
#     # on selectionne un nombre aléatoire de lignes
#     df_criticality_sample = df_criticality.sample(n=n)
#     # on stoke les index des lignes du df qui ne sont pas dans le sample
#     index_to_delete = df_criticality[~df_criticality.index.isin(df_criticality_sample.index)].index
#     # suppression des lignes exclu du sample
#     return df.drop(index_to_delete)

def balance_dataframe_by_criticality(df, criticality, n):
    # Filtre les lignes du df avec la valeur de criticality dans la colonne 'criticality_events'
    df_criticality = df[df['criticality_events'] == criticality]

    # Obtient le nombre de lignes dans df_criticality
    taille_population = df_criticality.shape[0]

    # Assurez-vous que n est inférieur ou égal à la taille de la population
    n = min(n, taille_population)

    # Sélectionne un nombre aléatoire de lignes
    df_criticality_sample = df_criticality.sample(n=n)

    # Stocke les index des lignes du df qui ne sont pas dans le sample
    index_to_delete = df_criticality[~df_criticality.index.isin(df_criticality_sample.index)].index

    # Suppression des lignes exclues du sample
    return df.drop(index_to_delete)


### par criticité UNDEFINED

In [18]:
metrics_df = balance_dataframe_by_criticality(metrics_df, CRITICALITY_NULL, errors)

### par criticité INFO

In [19]:
metrics_df = balance_dataframe_by_criticality(metrics_df, 'INFO', errors)

### par criticité WARNING

In [20]:
metrics_df = balance_dataframe_by_criticality(metrics_df, 'WARNING', errors)

In [21]:
# on vérifie la distribution par criticité
metrics_df['criticality_events'].value_counts()

ERROR       29395
INFO        29395
NO_EVENT    29395
WARNING     11384
Name: criticality_events, dtype: int64

# D. Création du datatset pour le pre-processing

In [22]:
metrics_reduced = metrics_df
metrics_reduced['jobId'] = 0
metrics_reduced.shape

(99569, 15)

## a) Association d'un jobId à une ligne metrics

In [23]:
# DataFrame 1 
# avec les intervalles de temps
dataframe1 = jobs_reduced[['started_at', 'ended_at', 'jobId']]
# conversion des colonnes au format datetime
cols_to_convert = ['started_at', 'ended_at']
dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)
dataframe1 = dataframe1.sort_values('started_at')
dataframe1 = dataframe1.reset_index()
dataframe1.head(3)

C:\Users\Allan\AppData\Local\Temp\ipykernel_16956\1041892598.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)


,index,started_at,ended_at,jobId
0,15113,2022-11-02 09:44:07.765000+00:00,2022-11-02 09:54:38.330000+00:00,1667382247
1,15114,2022-11-02 09:56:26.583000+00:00,2022-11-02 09:58:18.618000+00:00,1667382986
2,15115,2022-11-02 10:03:09.434000+00:00,2022-11-02 10:04:03.477000+00:00,1667383389


In [24]:
# DataFrame 2 
# avec les valeurs de date
dataframe2 = pd.DataFrame({
    'timestamp': metrics_reduced['timestamp'].values,
    'criticality_events': metrics_reduced['criticality_events'].values,
    'identification_events': metrics_reduced['identification_events'].values,
    'jobId': 0
}, index=None)
# conversion des colonnes au format datetime
dataframe2['timestamp'] = pd.to_datetime(dataframe2['timestamp'])
dataframe2 = dataframe2.sort_values('timestamp')
dataframe2 = dataframe2.reset_index()
dataframe2.head(3)

,index,timestamp,criticality_events,identification_events,jobId
0,0,2022-11-02 08:43:46.845000+00:00,ERROR,358,0
1,35269,2022-11-02 08:45:01.016000+00:00,WARNING,407,0
2,35270,2022-11-02 08:45:35.557000+00:00,NO_EVENT,0,0


In [25]:
import time
# début du temps d'execution de la cellule
start = time.time()

# Parcours chaque ligne de job
for index1, row1 in dataframe1.iterrows():
    # creation des variables
    job_starts = row1.started_at.tz_localize(None)
    job_ends = row1.ended_at.tz_localize(None)

    # mois de référence
    month = job_starts.month
    if month != job_ends.month :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) | (dataframe2['timestamp'].dt.month == job_ends.month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')
    else :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')

    for index2, row2 in dataframe2_by_month.iterrows():
        # creation des variables
        metrics_timestamp = row2['timestamp'].tz_localize(None)
        # si le timestamp de la lignes de row2 de metrics est compris dans l'intervalle de temps du job de row1
        if metrics_timestamp >= job_starts :
            if metrics_timestamp <= job_ends :
                dataframe2.loc[index2, 'jobId'] = dataframe1.loc[index1, 'jobId']
    # # avant de passer à la row1 suivante
    # # on affiche l'index1 toutes les 1000 lignes et son temps d'execution
    print(f'[ job index {index1} = time {time.time() - start}]') if index1 % 1000 == 0 and index1 != 0 else None


[ job index 1000 = time 494.37801456451416]
[ job index 2000 = time 661.142715215683]
[ job index 3000 = time 869.8050320148468]
[ job index 4000 = time 1162.0341975688934]
[ job index 5000 = time 1470.2898354530334]
[ job index 6000 = time 1920.6499609947205]
[ job index 7000 = time 2285.7571592330933]
[ job index 8000 = time 2769.8280436992645]
[ job index 9000 = time 3212.9434962272644]
[ job index 10000 = time 3588.4558939933777]
[ job index 11000 = time 4065.198122739792]
[ job index 12000 = time 4548.855000734329]
[ job index 13000 = time 5090.945028066635]
[ job index 14000 = time 5452.810800075531]
[ job index 15000 = time 5746.5716462135315]
[ job index 16000 = time 6112.002680301666]
[ job index 17000 = time 6410.903295755386]
[ job index 18000 = time 6802.935482978821]
[ job index 19000 = time 7092.161920309067]
[ job index 20000 = time 7413.628153085709]
[ job index 21000 = time 7673.116983652115]
[ job index 22000 = time 7955.641501665115]


In [26]:
# on vérifie la distribution par criticality_events pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
dataframe2[dataframe2['jobId'] != 0]['criticality_events'].value_counts()

INFO        17030
NO_EVENT    10440
ERROR        7414
WARNING      1114
Name: criticality_events, dtype: int64

In [27]:
print(f'Nombre de ligne de metrics : {dataframe2.shape[0]} (dont {dataframe2[dataframe2.jobId != 0].shape[0]} avec un jobId associé)')

Nombre de ligne de metrics : 99569 (dont 35998 avec un jobId associé)


## b) Fusion des données de job avec les lignes de metrics (dataframe2)

In [28]:
merged_df = jobs_reduced.merge(dataframe2, on='jobId')
merged_df.shape

(35998, 60)

La fusion on jobId permet de ne conserver que les lignes de metrics qui ont un jobId.

## c) Equilibrage des classes

In [29]:
# on détermine le nombre de lignes ERROR
merge_df_errors = merged_df['criticality_events'][merged_df['criticality_events'] == 'ERROR'].count()
print(f"Le nombre de lignes en criticité 'ERROR' : {merge_df_errors}")

Le nombre de lignes en criticité 'ERROR' : 7414


In [30]:
# réduction des autres classes par rapport à ERROR
merged_df = balance_dataframe_by_criticality(merged_df, CRITICALITY_NULL, merge_df_errors)
merged_df = balance_dataframe_by_criticality(merged_df, 'WARNING', merge_df_errors)
merged_df = balance_dataframe_by_criticality(merged_df, 'INFO', merge_df_errors)

In [31]:
# on vérifie la distribution par criticité pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
merged_df['criticality_events'].value_counts()

NO_EVENT    7414
ERROR       7414
INFO        7414
WARNING     1114
Name: criticality_events, dtype: int64

## d) Output

In [32]:
merged_df.to_csv(path_or_buf=save_csv)